In [1]:
! pip install statsmodels

You should consider upgrading via the 'c:\users\saidj\onedrive\documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('pruebillas.ipynb'), '..'))
os.chdir(root_dir)
sys.path.insert(0, os.path.join(root_dir, 'src'))

In [3]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply, barssince
import pandas as pd
import talib as ta
import numpy as np

from backbone.utils.general_purpose import diff_pips

In [4]:
symbols_path = './backbone/data/backtest/symbols/USDCAD.csv'
df = pd.read_csv(symbols_path)
df = df[['Date','Open','High','Low','Close', 'Volume']]
df

,Date,Open,High,Low,Close,Volume
0,2021-01-01 00:00:00,1.27316,1.27316,1.27316,1.27316,0.000000e+00
1,2021-01-01 00:01:00,1.27316,1.27316,1.27316,1.27316,0.000000e+00
2,2021-01-01 00:02:00,1.27316,1.27316,1.27316,1.27316,0.000000e+00
3,2021-01-01 00:03:00,1.27316,1.27316,1.27316,1.27316,0.000000e+00
4,2021-01-01 00:04:00,1.27316,1.27316,1.27316,1.27316,0.000000e+00
...,...,...,...,...,...,...
1578235,2024-01-01 23:55:00,1.32475,1.32475,1.32473,1.32473,2.400000e+06
1578236,2024-01-01 23:56:00,1.32474,1.32479,1.32455,1.32455,3.360000e+07
1578237,2024-01-01 23:57:00,1.32456,1.32463,1.32447,1.32447,2.965000e+07
1578238,2024-01-01 23:58:00,1.32449,1.32450,1.32446,1.32446,9.750000e+06


In [5]:
train_start = '2023-01-01'
train_end = '2023-03-01'

test_start = '2023-03-01'
test_end = '2023-04-01'

train_data = df[(df.Date > train_start) & (df.Date < train_end)]
test_data = df[(df.Date > test_start) & (df.Date < test_end)]

train_data.loc[:, 'Date'] = pd.to_datetime(train_data.Date)
test_data.loc[:, 'Date'] = pd.to_datetime(test_data.Date)

train_data = train_data.set_index('Date')
test_data = test_data.set_index('Date')


In [6]:
import numpy as np
import pandas as pd


def consecutive(close, open, n):
    # Determina la dirección de las velas: 1 para alcista, -1 para bajista, 0 para sin cambio
    direction = np.where(close > open, 1, np.where(close < open, -1, 0))
    
    # Detecta cambios en la dirección
    changes = np.diff(direction, prepend=direction[0])
    group = np.cumsum(changes != 0)
    
    # Contar el número de velas consecutivas dentro de cada grupo
    consecutive_candles = np.zeros_like(group)
    unique_groups, counts = np.unique(group, return_counts=True)
    for grp, count in zip(unique_groups, counts):
        consecutive_candles[group == grp] = np.arange(1, count + 1)
    
    # Crear una serie con valores 0 por defecto
    result = np.zeros_like(direction)
    
    # Asignar 1 o -1 para secuencias que tienen exactamente n velas consecutivas
    result[consecutive_candles == n] = direction[consecutive_candles == n]
    
    return list(result)



In [7]:
def  optim_func(series):
    return series['Return [%]'] / (series['Max. Drawdown [%]'])

In [8]:
from backtesting import Strategy

class ConsecutiveCandlesGrid(Strategy):
    n = 7
    percent_profit = 0.001
    percent_loss = 0.05
    pip_value = 0.0001
    grid_size = 15
    position_size = 3000
    rsi_down_threshold = 30
    rsi_up_threshold = 70
    max_minutes_in_trade = 400

    def init(self):
        self.consecutive = self.I(consecutive, self.data.Close, self.data.Open, n=self.n)
        self.rsi = self.I(ta.RSI, self.data.Close, 14)

    
    def next(self):
        close_prices = self.data.Close
        open_prices = self.data.Open
        actual_close_price = close_prices[-1]

        today = self.data.index[-1]
        today = today.tz_localize('UTC').tz_convert('UTC')  # Si el timestamp ya está en UTC pero no tiene la info

        # Verificar si hay posiciones abiertas
        if self.position:
            last_trade = self.trades[-1]
            first_trade = self.trades[0]

            profit_target = self.equity * self.percent_profit
            loss_target = self.equity * self.percent_loss

            total_profit = sum(trade.pl for trade in self.trades)
            
            last_trade_pl = last_trade.pl
            pips = diff_pips(last_trade.entry_price, actual_close_price, pip_value=self.pip_value, absolute=True)

            dinamic_grid_size = self.grid_size * (len(self.trades))

            time_in_position = (today - first_trade.entry_time.tz_localize('UTC').tz_convert('UTC'))
            time_in_position = time_in_position.total_seconds() // 60
            
            if time_in_position >= self.max_minutes_in_trade and total_profit >=0:
                self.position.close()

            elif total_profit <= -loss_target or total_profit >= profit_target:
                self.position.close()

            # Añadir posición en caso de que el profit/loss entre dentro del rango
            elif self.position.is_long and last_trade_pl < 0 and pips > dinamic_grid_size:
                self.buy(size=self.position_size)
                    
            elif self.position.is_short and last_trade_pl < 0 and pips > dinamic_grid_size:
                self.sell(size=self.position_size)

        else:

            # Abrir posición en función de las velas consecutivas 
            
            weekday = today.weekday()

            if self.consecutive == -1 and self.rsi > self.rsi_down_threshold and self.rsi < self.rsi_up_threshold and weekday < 4:
                self.buy(size=self.position_size)
                
            elif self.consecutive == 1 and self.rsi > self.rsi_down_threshold and self.rsi < self.rsi_up_threshold and weekday < 4:
                self.sell(size=self.position_size)


In [9]:

bt_train = Backtest(
    train_data, 
    ConsecutiveCandlesGrid, 
    cash=15000, 
    commission=0.0002,
    margin=1/30
)

stats = bt_train.optimize(
    n=[6, 7, 8, 9],
    percent_profit=[0.0005, 0.0008, 0.001],
    percent_loss=[0.01, 0.02, 0.03],
    grid_size=[10, 20, 25, 30],
    position_size=[2500, 3000, 3500, 4000],
    rsi_up_threshold=[55, 60, 70],
    rsi_down_threshold=[30, 40, 45],
    max_minutes_in_trade = [300, 400, 500],
    maximize='Return [%]',
    max_tries=500,
)

bt_train.plot(filename='./ConsecutiveCandlesGridTrain.html', resample=False)

stats

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 487 configurations.
  output = _optimize_grid()


  0%|          | 0/9 [00:00<?, ?it/s]

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.me

Start                     2023-01-01 00:00:00
End                       2023-02-28 23:59:00
Duration                     58 days 23:59:00
Exposure Time [%]                   50.885122
Equity Final [$]                 15717.142088
Equity Peak [$]                  15745.619736
Return [%]                           4.780947
Buy & Hold Return [%]                0.707713
Return (Ann.) [%]                   33.498188
Volatility (Ann.) [%]                6.136726
Sharpe Ratio                         5.458642
Sortino Ratio                       13.427708
Calmar Ratio                        33.847699
Max. Drawdown [%]                   -0.989674
Avg. Drawdown [%]                   -0.045129
Max. Drawdown Duration        4 days 17:01:00
Avg. Drawdown Duration        0 days 02:49:00
# Trades                                  220
Win Rate [%]                        64.090909
Best Trade [%]                       0.487269
Worst Trade [%]                     -0.284694
Avg. Trade [%]                    

In [19]:
print(f'n: {stats._strategy.n}')
print(f'percent_profit: {stats._strategy.percent_profit}')
print(f'percent_loss: {stats._strategy.percent_loss}')
print(f'grid_size: {stats._strategy.grid_size}')
print(f'position_size: {stats._strategy.position_size}')
print(f'rsi up: {stats._strategy.rsi_up_threshold}')
print(f'rsi down: {stats._strategy.rsi_down_threshold}')

n: 6
percent_profit: 0.001
percent_loss: 0.01
grid_size: 10
position_size: 4000
rsi up: 70
rsi down: 30


In [20]:
stats._trades.groupby(by=['ExitTime']).agg({'PnL':['sum','count']})

PnL      
                           sum count
ExitTime                            
2023-01-02 23:07:00   0.544480     3
2023-01-03 07:26:00  19.706384     3
2023-01-03 08:19:00  20.677136     1
2023-01-03 14:41:00  22.194688     4
2023-01-03 16:21:00  15.350616     1
...                        ...   ...
2023-02-27 10:18:00  -0.185184     3
2023-02-27 16:29:00   5.616072     4
2023-02-28 02:34:00   0.153688     1
2023-02-28 13:31:00   0.307088     2
2023-02-28 23:59:00 -21.991512     3

[99 rows x 2 columns]

In [21]:
# test_data['Date'] = pd.to_datetime(test_data['Date'])
# test_data = test_data.set_index('Date')
bt_test = Backtest(
    test_data, 
    ConsecutiveCandlesGrid, 
    cash=15000, 
    commission=0.0002,
    margin=1/30
)
 
test_stats = bt_test.run(
    n=6,
    percent_profit=0.0005, #stats._strategy.percent_profit,
    percent_loss=0.01, #stats._strategy.percent_loss,
    grid_size=10, #stats._strategy.grid_size,
    rsi_up_threshold=65, #stats._strategy.volume_threshold,
    rsi_down_threshold=40, #stats._strategy.volume_threshold,
    position_size=3500, #stats._strategy.position_size,
)


bt_test.plot(filename='./ConsecutiveCandlesGridTest.html', resample=False)

test_stats

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single forma

Start                     2023-03-01 00:00:00
End                       2023-03-31 23:59:00
Duration                     30 days 23:59:00
Exposure Time [%]                   39.119624
Equity Final [$]                 15010.395042
Equity Peak [$]                  15155.633835
Return [%]                             0.0693
Buy & Hold Return [%]               -0.924745
Return (Ann.) [%]                   -1.035013
Volatility (Ann.) [%]                4.754222
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -1.394325
Avg. Drawdown [%]                   -0.043641
Max. Drawdown Duration       17 days 05:04:00
Avg. Drawdown Duration        0 days 04:23:00
# Trades                                  100
Win Rate [%]                             69.0
Best Trade [%]                       0.374455
Worst Trade [%]                     -0.904401
Avg. Trade [%]                    

In [22]:

import plotly.express as px

y = test_stats._equity_curve.Equity
x = test_stats._equity_curve.index


fig = px.line(x=x, y=y)
fig.update_traces(textposition="bottom right")
fig.show()

In [23]:
trades = test_stats._trades.groupby(by=['ExitTime']).agg({'PnL':['sum','count'], 'Duration':'max'})
trades.columns = trades.columns.droplevel(0)
trades = trades.reset_index().rename(columns={'count':'ammount_trades'})
trades = trades.rename(columns={'sum':'profit'})
trades = trades.rename(columns={'max':'minutes_in_trade'})
trades

,ExitTime,profit,ammount_trades,minutes_in_trade
0,2023-03-01 09:02:00,8.566544,1,0 days 03:18:00
1,2023-03-01 13:25:00,7.546322,2,0 days 02:29:00
2,2023-03-01 21:40:00,7.551712,1,0 days 05:35:00
3,2023-03-02 08:00:00,8.699285,2,0 days 06:10:00
4,2023-03-02 15:02:00,7.718249,2,0 days 03:47:00
5,2023-03-02 19:02:00,8.515171,3,0 days 03:00:00
6,2023-03-06 02:42:00,8.174670,2,0 days 02:07:00
7,2023-03-06 17:07:00,0.012439,3,0 days 07:48:00
8,2023-03-07 08:34:00,2.897461,1,0 days 06:41:00
9,2023-03-08 01:12:00,1.732934,1,0 days 06:41:00


In [15]:
trades.minutes_in_trade.describe()


count                           49
mean     0 days 05:55:23.265306122
std      0 days 07:06:03.945752738
min                0 days 00:19:00
25%                0 days 01:01:00
50%                0 days 03:37:00
75%                0 days 06:57:00
max                1 days 10:15:00
Name: minutes_in_trade, dtype: object

In [16]:
# plot_heatmaps(heatmap, agg='mean')zzz

In [17]:
class ConsecutiveCandles(Strategy):
    n = 7
    stop_loss_pips = 10 # pips
    take_profit_pips = 10 # pips
    pip_value = 0.0001
    position_size = 0.001

    def init(self):
        pass
    
    def next(self):
        close_prices = self.data.Close
        open_prices = self.data.Open

        actual_close_price = close_prices[-1]

        if barssince(close_prices < open_prices) == self.n:
            sl_price = round(actual_close_price - self.stop_loss_pips * self.pip_value, 6)
            tp_price = round(actual_close_price + self.take_profit_pips * self.pip_value, 6)
            
            self.buy(sl=sl_price, tp=tp_price, size=self.position_size)

        elif barssince(close_prices > open_prices) == self.n:
            sl_price = round(actual_close_price + self.stop_loss_pips * self.pip_value, 6)
            tp_price = round(actual_close_price - self.take_profit_pips * self.pip_value, 6)
            # print(f'short tp:{tp_price}, actual {actual_close_price}, sl {sl_price}')

            self.sell(sl=sl_price, tp=tp_price, size=self.position_size)
